# vLLM Upstream vs vLLM ROCm
AMD maintains a vLLM fork: https://github.com/ROCm/vllm
Is this any better than the upstream https://github.com/vllm-project/vllm ?

```
# Clone
mamba create --name vllm-rocm --clone vllm
mamba activate vllm-rocm
python -m ipykernel install --user --name vllm-rocm

# Build (leave everything else the same as our previous setup)
git clone https://github.com/ROCm/vllm vllm-rocm
cd vllm-rocm
PYTORCH_ROCM_ARCH="gfx942" python setup.py develop

# vllm-rocm requirements
pip install -r requirements-rocm.txt
pip install msgspec pydantic

# without this you will get a `hipbsolidxgemm` error
cd gradlib
pip install .
cd ..
```


In [12]:
# Simple Benchmark
import time
import pandas as pd
import re

def benchmark_model(model, input_len, output_len, tp):    
    # Initialize the DataFrame
    df = pd.DataFrame(columns=['Requests per Second', 'Tokens per Second'])
    
    # Function to run the benchmark command and capture output
    def run_benchmark():
        command = f"VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len {input_len} --output-len {output_len} --model {model} -tp {tp}"
        # Run the command and capture the output
        start = time.time()
        output = get_ipython().getoutput(command)
        end = time.time()
        total_time = end-start
        output_str = ' '.join(output)
        print(f"  Run time: {total_time:.2f} seconds")
        # Use regular expressions to extract the throughput values
        matches = re.findall(r"Throughput:\s*([\d.]+)\s*requests/s,\s*([\d.]+)\s*tokens/s", output_str)
        if matches:
            requests_per_sec, tokens_per_sec = map(float, matches[0])
            return requests_per_sec, tokens_per_sec
        else:
            print(f"No throughput data found for {tuning} tuning.")
            return None, None

    # Run benchmarks for no GEMM Tuning
    none_rps, none_tps = run_benchmark()
    if none_rps is None or none_tps is None:
        print("Benchmark failed.")
        return None

    # Append No GEMM Tuning results to the DataFrame
    df.loc[len(df)] = {'Requests per Second': none_rps, 'Tokens per Second': none_tps}

    # Display the DataFrame
    print(df)
    return df

## vLLM Upstream

In [10]:
!VLLM_WORKER_MULTIPROC_METHOD='spawn' python -c 'import vllm; print(vllm.__version__)'

0.6.4.dev9+g5d264f4a


In [2]:
# Increase File handles
!ulimit -n 131072
!python vllm/collect_env.py

WARNING 10-29 10:22:31 rocm.py:13] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
PyTorch version: 2.6.0.dev20241015+rocm6.2
Is debug build: False
CUDA used to build PyTorch: N/A
ROCM used to build PyTorch: 6.2.41133-dd7f95766

OS: Ubuntu 22.04.5 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: Could not collect
CMake version: version 3.30.5
Libc version: glibc-2.35

Python version: 3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 18:08:57) [GCC 13.3.0] (64-bit runtime)
Python platform: Linux-6.8.0-47-generic-x86_64-with-glibc2.35
Is CUDA available: True
CUDA runtime version: Could not collect
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: AMD Instinct MI300X (gfx942:sramecc+:xnack-)
Nvidia driver version: Could not collect
cuDNN version: Could not collect
HIP runtime version: 6.2.41133
MIOpen runtime version: 3.2.0
Is XNNPACK available: True

CPU:
Architecture:                 

In [3]:
import pandas as pd
import re
import nbformat

In [13]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 4096, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8}
  Run time: 74.92 seconds
   Requests per Second  Tokens per Second
0                91.63            11728.8
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8}
  Run time: 88.60 seconds
   Requests per Second  Tokens per Second
0                43.82           11217.16
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8}
  Run time: 107.98 seconds
   Requests per Second  Tokens per Second
0                22.46           11497.47
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8}
  Run time: 159.48 seconds
   Requests per Second  Tokens per Second
0                10.94           11203.39
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8}
  Run time: 258.22 seconds
   Requests per Second  Tokens per Second
0                 5.23           1070

,Requests per Second,Tokens per Second,Config
0,91.63,11728.80,"input_len=0, output_len=128, tp=8"
1,43.82,11217.16,"input_len=0, output_len=256, tp=8"
2,22.46,11497.47,"input_len=0, output_len=512, tp=8"
3,10.94,11203.39,"input_len=0, output_len=1024, tp=8"
4,5.23,10702.26,"input_len=0, output_len=2048, tp=8"
5,2.19,8965.82,"input_len=0, output_len=4096, tp=8"


In [14]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 4096, 'output_len': 128, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8}
  Run time: 76.39 seconds
   Requests per Second  Tokens per Second
0                83.27           21318.35
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8}
  Run time: 77.30 seconds
   Requests per Second  Tokens per Second
0                73.55           28244.18
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8}
  Run time: 81.28 seconds
   Requests per Second  Tokens per Second
0                62.58           40052.48
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8}
  Run time: 86.43 seconds
   Requests per Second  Tokens per Second
0                45.13           51987.43
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8}
  Run time: 97.39 seconds
   Requests per Second  Tokens per Second
0                30.66        

,Requests per Second,Tokens per Second,Config
0,83.27,21318.35,"input_len=128, output_len=128, tp=8"
1,73.55,28244.18,"input_len=256, output_len=128, tp=8"
2,62.58,40052.48,"input_len=512, output_len=128, tp=8"
3,45.13,51987.43,"input_len=1024, output_len=128, tp=8"
4,30.66,66719.98,"input_len=2048, output_len=128, tp=8"
5,18.16,76695.09,"input_len=4096, output_len=128, tp=8"


In [15]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8}
  Run time: 77.59 seconds
   Requests per Second  Tokens per Second
0                 81.1           21247.29
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8}
  Run time: 298.27 seconds
   Requests per Second  Tokens per Second
0                 4.29           17156.23
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8}
  Run time: 305.56 seconds
   Requests per Second  Tokens per Second
0                 4.15           16997.87


,Requests per Second,Tokens per Second,Config
0,81.10,21247.29,"input_len=131, output_len=131, tp=8"
1,4.29,17156.23,"input_len=2000, output_len=2000, tp=8"
2,4.15,16997.87,"input_len=2048, output_len=2048, tp=8"


## vLLM ROCm
We switch the kernel here to `vllm-rocm` (in the top right pull-down) and run everything again.

In [6]:
%pushd /
!VLLM_WORKER_MULTIPROC_METHOD='spawn' python -c 'import vllm; print(vllm.__version__)'
%popd

/
0.6.4.dev5+g4bba0922
/mnt/nvme1n1p1/MI300-testing
popd -> /mnt/nvme1n1p1/MI300-testing


In [9]:
# Increase File handles
!ulimit -n 131072
!python ~/vllm-rocm/collect_env.py

WARNING 10-29 12:12:16 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
PyTorch version: 2.6.0.dev20241015+rocm6.2
Is debug build: False
CUDA used to build PyTorch: N/A
ROCM used to build PyTorch: 6.2.41133-dd7f95766

OS: Ubuntu 22.04.5 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: Could not collect
CMake version: version 3.30.5
Libc version: glibc-2.35

Python version: 3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 18:08:57) [GCC 13.3.0] (64-bit runtime)
Python platform: Linux-6.8.0-47-generic-x86_64-with-glibc2.35
Is CUDA available: True
CUDA runtime version: Could not collect
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: AMD Instinct MI300X (gfx942:sramecc+:xnack-)
Nvidia driver version: Could not collect
cuDNN version: Could not collect
HIP runtime version: 6.2.41133
MIOpen runtime version: 3.2.0
Is XNNPACK available: True

CPU:
Architecture:                 

In [3]:
import pandas as pd
import re
import nbformat

In [11]:
# Simple Benchmark
import time
import pandas as pd
import re

def benchmark_model(model, input_len, output_len, tp):    
    # Initialize the DataFrame
    df = pd.DataFrame(columns=['Requests per Second', 'Tokens per Second'])
    
    # Function to run the benchmark command and capture output
    def run_benchmark():
        command = f"VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len {input_len} --output-len {output_len} --model {model} -tp {tp}"
        # Run the command and capture the output
        start = time.time()
        output = get_ipython().getoutput(command)
        end = time.time()
        total_time = end-start
        output_str = ' '.join(output)
        print(f"  Run time: {total_time:.2f} seconds")
        # Use regular expressions to extract the throughput values
        matches = re.findall(r"Throughput:\s*([\d.]+)\s*requests/s,\s*([\d.]+)\s*tokens/s", output_str)
        if matches:
            requests_per_sec, tokens_per_sec = map(float, matches[0])
            return requests_per_sec, tokens_per_sec
        else:
            print(f"No throughput data found for {tuning} tuning.")
            return None, None

    # Run benchmarks for no GEMM Tuning
    none_rps, none_tps = run_benchmark()
    if none_rps is None or none_tps is None:
        print("Benchmark failed.")
        return None

    # Append No GEMM Tuning results to the DataFrame
    df.loc[len(df)] = {'Requests per Second': none_rps, 'Tokens per Second': none_tps}

    # Display the DataFrame
    print(df)
    return df

In [13]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 4096, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8}
  Run time: 81.67 seconds
   Requests per Second  Tokens per Second
0                85.87           10990.94
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8}
  Run time: 91.41 seconds
   Requests per Second  Tokens per Second
0                44.13           11296.39
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8}
  Run time: 112.58 seconds
   Requests per Second  Tokens per Second
0                22.52           11530.38
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8}
  Run time: 161.37 seconds
   Requests per Second  Tokens per Second
0                10.89           11154.05
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8}
  Run time: 257.02 seconds
   Requests per Second  Tokens per Second
0                  5.3           1086

,Requests per Second,Tokens per Second,Config
0,85.87,10990.94,"input_len=0, output_len=128, tp=8"
1,44.13,11296.39,"input_len=0, output_len=256, tp=8"
2,22.52,11530.38,"input_len=0, output_len=512, tp=8"
3,10.89,11154.05,"input_len=0, output_len=1024, tp=8"
4,5.30,10860.82,"input_len=0, output_len=2048, tp=8"
5,2.20,9026.35,"input_len=0, output_len=4096, tp=8"


In [14]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 4096, 'output_len': 128, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8}
  Run time: 80.26 seconds
   Requests per Second  Tokens per Second
0                 78.4           20070.07
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8}
  Run time: 83.16 seconds
   Requests per Second  Tokens per Second
0                69.67           26753.28
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8}
  Run time: 84.45 seconds
   Requests per Second  Tokens per Second
0                60.19           38520.14
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8}
  Run time: 89.50 seconds
   Requests per Second  Tokens per Second
0                44.31           51043.84
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8}
  Run time: 101.74 seconds
   Requests per Second  Tokens per Second
0                30.22       

,Requests per Second,Tokens per Second,Config
0,78.40,20070.07,"input_len=128, output_len=128, tp=8"
1,69.67,26753.28,"input_len=256, output_len=128, tp=8"
2,60.19,38520.14,"input_len=512, output_len=128, tp=8"
3,44.31,51043.84,"input_len=1024, output_len=128, tp=8"
4,30.22,65750.09,"input_len=2048, output_len=128, tp=8"
5,18.02,76106.02,"input_len=4096, output_len=128, tp=8"


In [12]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8}
  Run time: 82.60 seconds
   Requests per Second  Tokens per Second
0                 71.3            18679.5
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8}
  Run time: 300.54 seconds
   Requests per Second  Tokens per Second
0                 4.31           17234.79
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8}
  Run time: 307.76 seconds
   Requests per Second  Tokens per Second
0                 4.21           17257.16


,Requests per Second,Tokens per Second,Config
0,71.30,18679.50,"input_len=131, output_len=131, tp=8"
1,4.31,17234.79,"input_len=2000, output_len=2000, tp=8"
2,4.21,17257.16,"input_len=2048, output_len=2048, tp=8"


# 2024-10-30 Update
EmbeddedLM just published their article on MI300X best practices on the vLLM blog and at they suggested that I give the ROCm fork another try with all the tuning they did to see if the potentially more optimized ROCm kernels could overcome the CPU overhead issues:
- https://blog.vllm.ai/2024/10/23/vllm-serving-amd.html

In [17]:
  # disable automatic NUMA balancing
  !sudo sh -c 'echo 0 > /proc/sys/kernel/numa_balancing'
  # check if NUMA balancing is disabled (returns 0 if disabled)
  !cat /proc/sys/kernel/numa_balancing

0


In [21]:
# Default
!time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8


WARNING 10-30 14:10:50 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=128, output_len=128, model='meta-llama/Llama-3.1-8B-Instruct', tokenizer='meta-llama/Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=1, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-30 14:11:01 config.py:946] Defaulting to use mp for distributed inference
WARNING 10-30 14:11:01 a

In [20]:
# CK Flash Attention
!VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8

WARNING 10-30 14:09:26 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=128, output_len=128, model='meta-llama/Llama-3.1-8B-Instruct', tokenizer='meta-llama/Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=1, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-30 14:09:37 config.py:946] Defaulting to use mp for distributed inference
WARNING 10-30 14:09:37 a

In [19]:
# AMD Network Tuning
!NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8


WARNING 10-30 14:07:47 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=128, output_len=128, model='meta-llama/Llama-3.1-8B-Instruct', tokenizer='meta-llama/Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=1, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-30 14:07:57 config.py:946] Defaulting to use mp for distributed inference
WARNING 10-30 14:07:57 a

In [24]:
# Holy Hell, definitely leave Chunked Prefill Disabled
!NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8 --enable-chunked-prefill

WARNING 10-30 14:15:38 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=128, output_len=128, model='meta-llama/Llama-3.1-8B-Instruct', tokenizer='meta-llama/Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=1, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=True, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-30 14:15:49 config.py:946] Defaulting to use mp for distributed inference
INFO 10-30 14:15:49 confi

In [26]:
# Graph Capture size # Default 8192 - shouldn't make a difference for short context?
!NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8 --max-seq-len-to-capture 16384

WARNING 10-30 14:17:55 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
usage: benchmark_throughput.py [-h] [--backend {vllm,hf,mii}]
                               [--dataset DATASET] [--input-len INPUT_LEN]
                               [--output-len OUTPUT_LEN] [--model MODEL]
                               [--tokenizer TOKENIZER]
                               [--quantization {aqlm,awq,deepspeedfp,tpu_int8,fp8,fbgemm_fp8,modelopt,marlin,gguf,gptq_marlin_24,gptq_marlin,awq_marlin,gptq,compressed-tensors,bitsandbytes,qqq,experts_int8,neuron_quant,ipex,None}]
                               [--tensor-parallel-size TENSOR_PARALLEL_SIZE]
                               [--n N] [--num-prompts NUM_PROMPTS]
                               [--seed SEED]
                               [--hf-max-batch-size HF_MAX_BATCH_SIZE]
                               [--trust-remote-code]
                               [--max-model-len MAX_M

In [28]:
# Graph Capture size # Default 8192 - shouldn't make a difference for short context?
# Not a valid option? --max-seq-len-to-capture 16384
# !NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8

In [30]:
# Multi-step scheduling - should reduce GPU idle time
!NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8 --num-scheduler-steps 15

WARNING 10-30 14:21:32 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=128, output_len=128, model='meta-llama/Llama-3.1-8B-Instruct', tokenizer='meta-llama/Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=15, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-30 14:21:42 config.py:946] Defaulting to use mp for distributed inference
WARNING 10-30 14:21:42 

In [32]:
# Is this like --max-num-seqs? Let's see 1024 - nope.
!NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 128 --output-len 128 --model meta-llama/Llama-3.1-8B-Instruct -tp 8 --num-scheduler-steps 15 --max-num-batched-tokens 1024

WARNING 10-30 14:26:50 rocm.py:17] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=128, output_len=128, model='meta-llama/Llama-3.1-8B-Instruct', tokenizer='meta-llama/Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=15, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=1024, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-30 14:27:01 config.py:946] Defaulting to use mp for distributed inference
WARNING 10-30 14:27:01 

# OK, let's see how vLLM-ROCm does

In [34]:
# Simple Benchmark
import time
import pandas as pd
import re

def benchmark_model(model, input_len, output_len, tp):    
    # Initialize the DataFrame
    df = pd.DataFrame(columns=['Requests per Second', 'Tokens per Second'])
    
    # Function to run the benchmark command and capture output
    def run_benchmark():
        command = f"NCCL_MIN_NCHANNELS=112 VLLM_USE_TRITON_FLASH_ATTN=0 time python vllm/benchmarks/benchmark_throughput.py --backend vllm  --num-scheduler-steps 15 --input-len {input_len} --output-len {output_len} --model {model} -tp {tp}"
        # Run the command and capture the output
        start = time.time()
        output = get_ipython().getoutput(command)
        end = time.time()
        total_time = end-start
        output_str = ' '.join(output)
        print(f"  Run time: {total_time:.2f} seconds")
        # Use regular expressions to extract the throughput values
        matches = re.findall(r"Throughput:\s*([\d.]+)\s*requests/s,\s*([\d.]+)\s*tokens/s", output_str)
        if matches:
            requests_per_sec, tokens_per_sec = map(float, matches[0])
            return requests_per_sec, tokens_per_sec
        else:
            print(f"No throughput data found for {tuning} tuning.")
            return None, None

    # Run benchmarks for no GEMM Tuning
    none_rps, none_tps = run_benchmark()
    if none_rps is None or none_tps is None:
        print("Benchmark failed.")
        return None

    # Append No GEMM Tuning results to the DataFrame
    df.loc[len(df)] = {'Requests per Second': none_rps, 'Tokens per Second': none_tps}

    # Display the DataFrame
    print(df)
    return df

In [35]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 4096, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8}
  Run time: 75.56 seconds
   Requests per Second  Tokens per Second
0               174.39           22321.68
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8}
  Run time: 82.09 seconds
   Requests per Second  Tokens per Second
0                89.95           23026.23
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8}
  Run time: 90.89 seconds
   Requests per Second  Tokens per Second
0                45.38           23236.72
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8}
  Run time: 114.40 seconds
   Requests per Second  Tokens per Second
0                22.32           22856.67
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8}
  Run time: 167.86 seconds
   Requests per Second  Tokens per Second
0                10.14           20763

,Requests per Second,Tokens per Second,Config
0,174.39,22321.68,"input_len=0, output_len=128, tp=8"
1,89.95,23026.23,"input_len=0, output_len=256, tp=8"
2,45.38,23236.72,"input_len=0, output_len=512, tp=8"
3,22.32,22856.67,"input_len=0, output_len=1024, tp=8"
4,10.14,20763.64,"input_len=0, output_len=2048, tp=8"
5,3.51,14394.31,"input_len=0, output_len=4096, tp=8"


In [36]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 4096, 'output_len': 128, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8}
  Run time: 77.26 seconds
   Requests per Second  Tokens per Second
0               146.46           37494.08
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8}
  Run time: 78.68 seconds
   Requests per Second  Tokens per Second
0               122.85           47174.45
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8}
  Run time: 78.70 seconds
   Requests per Second  Tokens per Second
0                94.51           60489.33
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8}
  Run time: 85.56 seconds
   Requests per Second  Tokens per Second
0                64.38           74161.48
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8}
  Run time: 94.52 seconds
   Requests per Second  Tokens per Second
0                38.88        

,Requests per Second,Tokens per Second,Config
0,146.46,37494.08,"input_len=128, output_len=128, tp=8"
1,122.85,47174.45,"input_len=256, output_len=128, tp=8"
2,94.51,60489.33,"input_len=512, output_len=128, tp=8"
3,64.38,74161.48,"input_len=1024, output_len=128, tp=8"
4,38.88,84612.11,"input_len=2048, output_len=128, tp=8"
5,21.06,88950.39,"input_len=4096, output_len=128, tp=8"


In [37]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8}
  Run time: 76.49 seconds
   Requests per Second  Tokens per Second
0               144.87           37957.05
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8}
  Run time: 202.68 seconds
   Requests per Second  Tokens per Second
0                 7.53           30123.15
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8}
  Run time: 205.52 seconds
   Requests per Second  Tokens per Second
0                 7.39           30269.69


,Requests per Second,Tokens per Second,Config
0,144.87,37957.05,"input_len=131, output_len=131, tp=8"
1,7.53,30123.15,"input_len=2000, output_len=2000, tp=8"
2,7.39,30269.69,"input_len=2048, output_len=2048, tp=8"


# Now Let's try vLLM again
- We switch back the the vllm kernel

In [1]:
  # disable automatic NUMA balancing
  !sudo sh -c 'echo 0 > /proc/sys/kernel/numa_balancing'
  # check if NUMA balancing is disabled (returns 0 if disabled)
  !cat /proc/sys/kernel/numa_balancing

0


In [2]:
# Simple Benchmark
import time
import pandas as pd
import re

def benchmark_model(model, input_len, output_len, tp):    
    # Initialize the DataFrame
    df = pd.DataFrame(columns=['Requests per Second', 'Tokens per Second'])
    
    # Function to run the benchmark command and capture output
    def run_benchmark():
        # Run the command and capture the output
        start = time.time()
        output = get_ipython().getoutput(command)
        end = time.time()
        total_time = end-start
        output_str = ' '.join(output)
        print(f"  Run time: {total_time:.2f} seconds")
        # Use regular expressions to extract the throughput values
        matches = re.findall(r"Throughput:\s*([\d.]+)\s*requests/s,\s*([\d.]+)\s*tokens/s", output_str)
        if matches:
            requests_per_sec, tokens_per_sec = map(float, matches[0])
            return requests_per_sec, tokens_per_sec
        else:
            print(f"No throughput data found for {tuning} tuning.")
            return None, None

    # Run benchmarks for no GEMM Tuning
    none_rps, none_tps = run_benchmark()
    if none_rps is None or none_tps is None:
        print("Benchmark failed.")
        return None

    # Append No GEMM Tuning results to the DataFrame
    df.loc[len(df)] = {'Requests per Second': none_rps, 'Tokens per Second': none_tps}

    # Display the DataFrame
    print(df)
    return df

In [3]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 4096, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8}
  Run time: 71.67 seconds
   Requests per Second  Tokens per Second
0               167.16            21396.5
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8}
  Run time: 79.76 seconds
   Requests per Second  Tokens per Second
0                84.53           21638.64
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8}
  Run time: 89.27 seconds
   Requests per Second  Tokens per Second
0                42.96           21993.74
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8}
  Run time: 112.99 seconds
   Requests per Second  Tokens per Second
0                21.39           21907.37
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8}
  Run time: 169.98 seconds
   Requests per Second  Tokens per Second
0                 9.75           19976

,Requests per Second,Tokens per Second,Config
0,167.16,21396.50,"input_len=0, output_len=128, tp=8"
1,84.53,21638.64,"input_len=0, output_len=256, tp=8"
2,42.96,21993.74,"input_len=0, output_len=512, tp=8"
3,21.39,21907.37,"input_len=0, output_len=1024, tp=8"
4,9.75,19976.14,"input_len=0, output_len=2048, tp=8"
5,3.54,14485.69,"input_len=0, output_len=4096, tp=8"


In [4]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 4096, 'output_len': 128, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8}
  Run time: 72.80 seconds
   Requests per Second  Tokens per Second
0               139.63           35745.19
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8}
  Run time: 74.33 seconds
   Requests per Second  Tokens per Second
0               119.27           45797.94
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8}
  Run time: 76.69 seconds
   Requests per Second  Tokens per Second
0                92.25           59037.91
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8}
  Run time: 82.70 seconds
   Requests per Second  Tokens per Second
0                62.74           72281.77
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8}
  Run time: 91.09 seconds
   Requests per Second  Tokens per Second
0                38.23        

,Requests per Second,Tokens per Second,Config
0,139.63,35745.19,"input_len=128, output_len=128, tp=8"
1,119.27,45797.94,"input_len=256, output_len=128, tp=8"
2,92.25,59037.91,"input_len=512, output_len=128, tp=8"
3,62.74,72281.77,"input_len=1024, output_len=128, tp=8"
4,38.23,83190.57,"input_len=2048, output_len=128, tp=8"
5,21.15,89324.95,"input_len=4096, output_len=128, tp=8"


In [5]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 131, 'output_len': 131, 'tp': 8}
  Run time: 73.34 seconds
   Requests per Second  Tokens per Second
0               138.15           36195.75
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2000, 'output_len': 2000, 'tp': 8}
  Run time: 201.68 seconds
   Requests per Second  Tokens per Second
0                 7.34           29374.21
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 2048, 'tp': 8}
  Run time: 205.29 seconds
   Requests per Second  Tokens per Second
0                 7.17           29378.71


,Requests per Second,Tokens per Second,Config
0,138.15,36195.75,"input_len=131, output_len=131, tp=8"
1,7.34,29374.21,"input_len=2000, output_len=2000, tp=8"
2,7.17,29378.71,"input_len=2048, output_len=2048, tp=8"
